# **Chapter 3: Blockchain Architecture and Data Structures**

---

## **3.1 Anatomy of a Block**

### **3.1.1 Block Header Components**

A block in a blockchain consists of two main parts: the **header** and the **body**. The header contains metadata about the block and is what gets hashed to create the block's unique identifier.

```
Block Structure Overview:

┌─────────────────────────────────────────────────────────────────┐
│                         BLOCK                                   │
│                   (Container of Data)                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │                    BLOCK HEADER                          │   │
│  │                   (Metadata Layer)                       │   │
│  │                                                         │   │
│  │  ┌─────────────────────────────────────────────────┐   │   │
│  │  │ Previous Block Hash    (32 bytes)               │   │   │
│  │  │ Links to parent block                           │   │   │
│  │  └─────────────────────────────────────────────────┘   │   │
│  │                                                         │   │
│  │  ┌─────────────────────────────────────────────────┐   │   │
│  │  │ Merkle Root            (32 bytes)               │   │   │
│  │  │ Root hash of transaction tree                   │   │   │
│  │  └─────────────────────────────────────────────────┘   │   │
│  │                                                         │   │
│  │  ┌─────────────────────────────────────────────────┐   │   │
│  │  │ Timestamp              (4 bytes)                │   │   │
│  │  │ Unix timestamp of block creation                │   │   │
│  │  └─────────────────────────────────────────────────┘   │   │
│  │                                                         │   │
│  │  ┌─────────────────────────────────────────────────┐   │   │
│  │  │ Difficulty Target      (4 bytes)                │   │   │
│  │  │ Mining difficulty for this block                │   │   │
│  │  └─────────────────────────────────────────────────┘   │   │
│  │                                                         │   │
│  │  ┌─────────────────────────────────────────────────┐   │   │
│  │  │ Nonce                  (4 bytes)              │   │   │
│  │  │ Number used once (mining)                     │   │   │
│  │  └─────────────────────────────────────────────────┘   │   │
│  │                                                         │   │
│  │  ┌─────────────────────────────────────────────────┐   │   │
│  │  │ Block Version          (4 bytes)                │   │   │
│  │  │ Protocol version                                  │   │   │
│  │  └─────────────────────────────────────────────────┘   │   │
│  │                                                         │   │
│  │  Total Header Size: ~80 bytes (Bitcoin)                 │   │
│  │                                                         │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │                    BLOCK BODY                            │   │
│  │                  (Transaction Data)                      │   │
│  │                                                         │   │
│  │  ┌─────────────────────────────────────────────────┐   │   │
│  │  │ Transaction Count      (Variable)               │   │   │
│  │  │ Number of transactions in block                 │   │   │
│  │  └─────────────────────────────────────────────────┘   │   │
│  │                                                         │   │
│  │  ┌─────────────────────────────────────────────────┐   │   │
│  │  │ Transactions           (Variable size)          │   │   │
│  │  │ List of all transactions                        │   │   │
│  │  │ • Transaction 1                                 │   │   │
│  │  │ • Transaction 2                                 │   │   │
│  │  │ • Transaction 3                                 │   │   │
│  │  │ • ...                                           │   │   │
│  │  │ • Transaction N                                 │   │   │
│  │  └─────────────────────────────────────────────────┘   │   │
│  │                                                         │   │
│  │  Typical Size: 1-2 MB (varies by blockchain)            │   │
│  │                                                         │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Detailed Header Fields:**

| Field | Size | Purpose |
|-------|------|---------|
| **Previous Block Hash** | 32 bytes | Cryptographic link to parent block, ensures chain integrity |
| **Merkle Root** | 32 bytes | Hash of all transactions; allows efficient verification |
| **Timestamp** | 4 bytes | Unix timestamp; prevents replay attacks, enables difficulty adjustment |
| **Difficulty Target** | 4 bytes | Encoded target hash value; determines mining difficulty |
| **Nonce** | 4 bytes | Miner-adjustable value; used to find valid block hash |
| **Version** | 4 bytes | Protocol version; enables soft forks and upgrades |

**Why the Header is Separate:**

The header is kept small (80 bytes in Bitcoin) for efficiency:
- Light clients only need headers (not full blocks)
- SPV (Simplified Payment Verification) uses headers
- Headers can be processed quickly for validation

### **3.1.2 Block Body and Transactions**

The block body contains the actual transaction data that the block is confirming.

```
Block Body Structure:

┌─────────────────────────────────────────────────────────────────┐
│                    BLOCK BODY                                   │
│                                                                 │
│  Transaction Counter (VarInt)                                  │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Number of transactions (1-9 bytes, compact encoding)      │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
│  Transaction List                                                │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │                                                         │   │
│  │  Transaction 1 (Coinbase in Bitcoin)                    │   │
│  │  ┌─────────────────────────────────────────────────┐   │   │
│  │  │ Version              (4 bytes)                  │   │   │
│  │  │ Input Count          (VarInt)                   │   │   │
│  │  │ Inputs               (References + Scripts)     │   │   │
│  │  │ Output Count         (VarInt)                   │   │   │
│  │  │ Outputs              (Addresses + Amounts)      │   │   │
│  │  │ Lock Time            (4 bytes)                  │   │   │
│  │  └─────────────────────────────────────────────────┘   │   │
│  │                                                         │   │
│  │  Transaction 2                                          │   │
│  │  ┌─────────────────────────────────────────────────┐   │   │
│  │  │ Version              (4 bytes)                  │   │   │
│  │  │ Input Count          (VarInt)                   │   │   │
│  │  │ Inputs               (References to UTXOs)      │   │   │
│  │  │ Output Count         (VarInt)                   │   │   │
│  │  │ Outputs              (New UTXOs)                │   │   │
│  │  │ Lock Time            (4 bytes)                  │   │   │
│  │  └─────────────────────────────────────────────────┘   │   │
│  │                                                         │   │
│  │  [More transactions...]                                 │   │
│  │                                                         │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
│  Size Constraints:                                              │
│  • Bitcoin: 1 MB to 4 MB (with SegWit)                         │
│  • Ethereum: ~30 million gas limit (variable)                  │
│  • Block size affects propagation time and decentralization      │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Transaction Size Calculation:**

```
Transaction Size Formula (Bitcoin):

Size = 4 (version) 
     + VarInt(inputs) 
     + Σ(InputSize) 
     + VarInt(outputs) 
     + Σ(OutputSize) 
     + 4 (locktime)

Typical simple transaction:
• 1 input, 2 outputs ≈ 250 bytes
• 10 inputs, 2 outputs ≈ 1,500 bytes

Example breakdown:
┌────────────────────────────────────────┐
│ Version:           4 bytes             │
│ Input count:       1 byte              │
│ Input 1:           148 bytes           │
│   - Prev TX hash:  32 bytes            │
│   - Prev TX index: 4 bytes             │
│   - Script length: 1 byte              │
│   - ScriptSig:     ~107 bytes          │
│   - Sequence:      4 bytes             │
│ Output count:      1 byte              │
│ Output 1:          34 bytes            │
│ Output 2:          34 bytes            │
│ Lock time:         4 bytes             │
├────────────────────────────────────────┤
│ Total:             ~250 bytes          │
└────────────────────────────────────────┘
```

### **3.1.3 Genesis Block: The First Block**

The genesis block is the first block in any blockchain. It has special properties and is hardcoded into the protocol.

```
Genesis Block Characteristics:

┌─────────────────────────────────────────────────────────────────┐
│                    GENESIS BLOCK                                │
│                                                                 │
│  Special Properties:                                            │
│  ───────────────────                                            │
│  • Block Height: 0                                              │
│  • Previous Hash: All zeros (0x0000...0000)                    │
│  • No parent block exists                                       │
│  • Hardcoded into software                                      │
│  • Cannot be changed without forking entire chain              │
│  • Often contains message from creator                          │
│                                                                 │
│  Bitcoin Genesis Block (January 3, 2009):                      │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Previous Hash: 00000000000000000000000000000000...      │   │
│  │ Merkle Root:   4a5e1e4baab89f3a32518a88c31bc87f...      │   │
│  │ Timestamp:     1231006505 (2009-01-03 18:15:05 GMT)     │   │
│  │ Difficulty:    1 (minimum difficulty)                    │   │
│  │ Nonce:         2083236893                                │   │
│  │                                                          │   │
│  │ Coinbase Transaction Message:                            │   │
│  │ "The Times 03/Jan/2009 Chancellor on brink of          │   │
│  │  second bailout for banks"                               │   │
│  │                                                          │   │
│  │ This proves the block wasn't mined before Jan 3, 2009   │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
│  Why Genesis Block Matters:                                     │
│  ───────────────────────────                                    │
│  • Anchor for entire blockchain                                │
│  • Starting point for all validation                         │
│  • Contains initial parameters                                │
│  • Reference point for difficulty calculations                │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **3.1.4 Block Hash Calculation**

The block hash is calculated by hashing the header twice (in Bitcoin) or once (in Ethereum).

```
Block Hash Calculation Process:

┌─────────────────────────────────────────────────────────────────┐
│               BITCOIN BLOCK HASH (Double SHA-256)              │
│                                                                 │
│  Step 1: Serialize the Block Header                            │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Concatenate header fields in order:                     │   │
│  │                                                         │   │
│  │ [Version(4)] + [PrevHash(32)] + [MerkleRoot(32)] +     │   │
│  │ [Timestamp(4)] + [Difficulty(4)] + [Nonce(4)]          │   │
│  │                                                         │   │
│  │ Result: 80 bytes of binary data                          │   │
│  └─────────────────────────────────────────────────────────┘   │
│                           │                                     │
│                           ▼                                     │
│  Step 2: First SHA-256 Hash                                   │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ SHA-256(Header) = IntermediateHash                      │   │
│  │                                                         │   │
│  │ Result: 32 bytes (256 bits)                              │   │
│  └─────────────────────────────────────────────────────────┘   │
│                           │                                     │
│                           ▼                                     │
│  Step 3: Second SHA-256 Hash                                  │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ SHA-256(IntermediateHash) = BlockHash                   │   │
│  │                                                         │   │
│  │ Result: Final 32-byte Block Hash                         │   │
│  │ Example: 000000000019d6689c085ae165831e934ff763...      │   │
│  │          (Must start with certain number of zeros)        │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
│  Why Double Hash?                                               │
│  ─────────────────                                              │
│  • Protection against length-extension attacks                 │
│  • Additional security margin                                  │
│  • Historical design choice                                    │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Code Example: Block Hash Calculation**

```javascript
// Block Hash Calculation Example
const crypto = require('crypto');

/**
 * Calculate Bitcoin-style block hash (double SHA-256)
 * @param {Object} header - Block header fields
 * @returns {string} - Block hash in hex
 */
function calculateBlockHash(header) {
    // Step 1: Serialize header fields
    // In real Bitcoin, this uses specific byte ordering (little-endian)
    
    const version = Buffer.alloc(4);
    version.writeUInt32LE(header.version);
    
    const prevHash = Buffer.from(header.previousHash, 'hex').reverse(); // Bitcoin uses reversed byte order
    
    const merkleRoot = Buffer.from(header.merkleRoot, 'hex').reverse();
    
    const timestamp = Buffer.alloc(4);
    timestamp.writeUInt32LE(header.timestamp);
    
    const bits = Buffer.alloc(4);
    bits.writeUInt32LE(header.difficultyBits);
    
    const nonce = Buffer.alloc(4);
    nonce.writeUInt32LE(header.nonce);
    
    // Concatenate all fields
    const headerBuffer = Buffer.concat([
        version,
        prevHash,
        merkleRoot,
        timestamp,
        bits,
        nonce
    ]);
    
    console.log('Serialized Header (hex):', headerBuffer.toString('hex'));
    console.log('Header Size:', headerBuffer.length, 'bytes');
    
    // Step 2: First SHA-256
    const hash1 = crypto.createHash('sha256').update(headerBuffer).digest();
    console.log('First Hash:', hash1.toString('hex'));
    
    // Step 3: Second SHA-256
    const hash2 = crypto.createHash('sha256').update(hash1).digest();
    console.log('Final Hash (reversed for display):', hash2.reverse().toString('hex'));
    
    return hash2.reverse().toString('hex'); // Reverse for display (Bitcoin convention)
}

// Example: Create a block header
const blockHeader = {
    version: 1,
    previousHash: '0000000000000000000000000000000000000000000000000000000000000000', // Genesis
    merkleRoot: '4a5e1e4baab89f3a32518a88c31bc87f618f76673e2cc77ab2127b7afdeda33b',
    timestamp: 1231006505, // Jan 3, 2009
    difficultyBits: 0x1d00ffff,
    nonce: 2083236893
};

console.log('=== BLOCK HASH CALCULATION ===\n');
const hash = calculateBlockHash(blockHeader);
console.log('\nCalculated Block Hash:', hash);

// Verify it matches Bitcoin genesis block hash
const genesisHash = '000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f';
console.log('Expected Genesis Hash:', genesisHash);
console.log('Match:', hash === genesisHash ? '✓ YES' : '✗ NO');

/*
Note: This is a simplified version. Real Bitcoin uses specific 
serialization rules and byte ordering that would require additional 
implementation details to perfectly match.
*/
```

---

## **3.2 The Chain Structure**

### **3.2.1 Linking Blocks Together**

Blocks are linked together through cryptographic hashes, creating an immutable chain.

```
How Blocks Link Together:

┌─────────────────────────────────────────────────────────────────┐
│                     BLOCKCHAIN LINKAGE                          │
│                                                                 │
│  Block 0 (Genesis)                                              │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Hash: 000000000019d6... (Block 0 Hash)                 │   │
│  │ Prev: 00000000000000... (All zeros)                     │   │
│  │ Data: [Genesis Transactions]                           │   │
│  └─────────────────────────────────────────────────────────┘   │
│                              │                                  │
│                              │ Hash of Block 0 becomes          │
│                              │ Previous Hash of Block 1         │
│                              ▼                                  │
│  Block 1                                                        │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Hash: 00000000839a8e... (Block 1 Hash)                 │   │
│  │ Prev: 000000000019d6... (Hash of Block 0) ←────────────│───┤
│  │ Data: [Transactions]                                   │   │
│  └─────────────────────────────────────────────────────────┘   │
│                              │                                  │
│                              │ Hash of Block 1 becomes          │
│                              │ Previous Hash of Block 2         │
│                              ▼                                  │
│  Block 2                                                        │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Hash: 000000006c02c8... (Block 2 Hash)                 │   │
│  │ Prev: 00000000839a8e... (Hash of Block 1) ←────────────│───┤
│  │ Data: [Transactions]                                   │   │
│  └─────────────────────────────────────────────────────────┘   │
│                              │                                  │
│                              ▼                                  │
│                            ...                                  │
│                              │                                  │
│                              ▼                                  │
│  Block N (Latest)                                               │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Hash: 00000000000004... (Block N Hash)                 │   │
│  │ Prev: 00000000000003... (Hash of Block N-1)            │   │
│  │ Data: [Latest Transactions]                            │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
│  Critical Property:                                             │
│  If you change ANY data in Block 0, its hash changes.          │
│  This makes Block 1's "Prev" pointer invalid.                  │
│  This invalidates Block 1's hash, making Block 2 invalid, etc.  │
│  The entire chain breaks!                                        │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **3.2.2 Previous Hash and Chain Integrity**

The `previousHash` field is what makes the blockchain immutable and ordered.

```
Chain Integrity Mechanism:

┌─────────────────────────────────────────────────────────────────┐
│               WHY THE CHAIN IS IMMUTABLE                        │
│                                                                 │
│  Scenario: Attacker wants to change a transaction              │
│  in Block 1.                                                    │
│                                                                 │
│  Original Chain:                                                │
│  ┌──────────┐    ┌──────────┐    ┌──────────┐                │
│  │ Block 0  │───▶│ Block 1  │───▶│ Block 2  │                │
│  │ Hash: A  │    │ Hash: B  │    │ Hash: C  │                │
│  │ Prev: 0  │    │ Prev: A  │    │ Prev: B  │                │
│  └──────────┘    └──────────┘    └──────────┘                │
│                                                                 │
│  Step 1: Change transaction in Block 1                          │
│  ┌──────────┐    ┌──────────┐    ┌──────────┐                │
│  │ Block 0  │    │ Block 1  │◀── Modified!                   │
│  │ Hash: A  │    │ Hash: B' │    │ Hash: C  │                │
│  │ Prev: 0  │    │ Prev: A  │    │ Prev: B  │◀── Invalid!    │
│  └──────────┘    └──────────┘    └──────────┘                │
│       │               │               │                        │
│       │               │               │                        │
│       │               │               │                        │
│       │          B ≠ B' (hash changed)                         │
│       │               │               │                        │
│       │               └──────┬──────┘                        │
│       │                      │                                 │
│       │                 B' ≠ B                                 │
│       │                 (Block 2's prev hash                    │
│       │                  no longer matches)                     │
│                                                                 │
│  Step 2: Must recalculate Block 1's hash                       │
│  Result: Hash changes from B to B'                             │
│                                                                 │
│  Step 3: Block 2 now has wrong previous hash                     │
│  Block 2 says Prev: B, but Block 1 now has Hash: B'            │
│                                                                 │
│  Step 4: Must recalculate Block 2 with new prev hash           │
│  Result: Block 2's hash changes from C to C'                   │
│                                                                 │
│  Step 5: Must recalculate Block 3, 4, 5... ALL subsequent blocks│
│                                                                 │
│  Cost: To change one old block, must redo ALL work since then!  │
│  This is computationally infeasible for established chains.      │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **3.2.3 Block Height and Depth**

Block height is a measure of how many blocks precede a given block in the chain.

```
Block Height and Depth Concepts:

┌─────────────────────────────────────────────────────────────────┐
│                    BLOCK HEIGHT                                 │
│                                                                 │
│  Genesis Block:                                                 │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Height: 0                                                │   │
│  │ Depth: N (where N is current chain length)               │   │
│  │ Confirmations: N+1                                       │   │
│  └─────────────────────────────────────────────────────────┘   │
│                              │                                  │
│                              ▼                                  │
│  Block 1:                                                       │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Height: 1                                                │   │
│  │ Depth: N-1                                               │   │
│  │ Confirmations: N                                         │   │
│  └─────────────────────────────────────────────────────────┘   │
│                              │                                  │
│                              ▼                                  │
│  Block 2:                                                       │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Height: 2                                                │   │
│  │ Depth: N-2                                               │   │
│  │ Confirmations: N-1                                       │   │
│  └─────────────────────────────────────────────────────────┘   │
│                              │                                  │
│                              ▼                                  │
│                             ...                                 │
│                              │                                  │
│                              ▼                                  │
│  Block N (Latest):                                              │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Height: N                                                │   │
│  │ Depth: 0                                                 │   │
│  │ Confirmations: 1                                         │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
│  Definitions:                                                   │
│  ────────────                                                   │
│  • Height: Number of blocks preceding this block              │
│  • Depth: Number of blocks following this block               │
│  • Confirmations: How many blocks have been mined on top       │
│                   (Height of latest - Height of this + 1)      │
│                                                                 │
│  Why Confirmations Matter:                                      │
│  ─────────────────────────                                      │
│  0 confirmations: Transaction in mempool, not mined yet       │
│  1 confirmation: In latest block (could still be orphaned)    │
│  6+ confirmations: Generally considered secure (Bitcoin)      │
│  12+ confirmations: Very secure, virtually irreversible         │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## **3.3 Transactions**

### **3.3.1 Transaction Structure**

A transaction is a data structure that records the transfer of value or execution of logic.

```
Generic Transaction Structure:

┌─────────────────────────────────────────────────────────────────┐
│                    TRANSACTION STRUCTURE                        │
│                                                                 │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Version          (4 bytes)                              │   │
│  │ Indicates transaction format version                    │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Input Count      (VarInt, 1-9 bytes)                    │   │
│  │ Number of inputs referencing previous outputs          │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Inputs           (Variable)                             │   │
│  │ ┌─────────────────────────────────────────────────────┐│   │
│  │ │ Input 1                                             ││   │
│  │ │ • Previous TX Hash (32 bytes)                       ││   │
│  │ │ • Previous Output Index (4 bytes)                   ││   │
│  │ │ • Script Length (VarInt)                            ││   │
│  │ │ • Script (Signature + Public Key)                   ││   │
│  │ │ • Sequence Number (4 bytes)                         ││   │
│  │ └─────────────────────────────────────────────────────┘│   │
│  │ ┌─────────────────────────────────────────────────────┐│   │
│  │ │ Input 2                                             ││   │
│  │ │ [Same structure...]                                 ││   │
│  │ └─────────────────────────────────────────────────────┘│   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Output Count     (VarInt, 1-9 bytes)                    │   │
│  │ Number of outputs created by this transaction          │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Outputs          (Variable)                             │   │
│  │ ┌─────────────────────────────────────────────────────┐│   │
│  │ │ Output 1                                            ││   │
│  │ │ • Value/Amount (8 bytes)                              ││   │
│  │ │ • Script Length (VarInt)                              ││   │
│  │ │ • Script (Locking conditions)                       ││   │
│  │ └─────────────────────────────────────────────────────┘│   │
│  │ ┌─────────────────────────────────────────────────────┐│   │
│  │ │ Output 2                                            ││   │
│  │ │ [Same structure...]                                 ││   │
│  │ └─────────────────────────────────────────────────────┘│   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Lock Time        (4 bytes)                              │   │
│  │ Earliest time/block when transaction can be mined      │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
└────────────────────────────────────────────────────────────────────────┘

### **3.3.2 Inputs and Outputs (UTXO Model)**

Bitcoin uses the UTXO (Unspent Transaction Output) model. This is fundamentally different from traditional accounting.

```
UTXO Model Explained:

┌─────────────────────────────────────────────────────────────────┐
│                    UTXO CONCEPT                                 │
│                                                                 │
│  Traditional Banking (Account Model):                            │
│  ───────────────────────────────────                            │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Alice's Account: $100                                    │   │
│  │ Bob's Account: $50                                       │   │
│  │                                                          │   │
│  │ Alice sends $30 to Bob:                                  │   │
│  │   Alice: $100 → $70                                      │   │
│  │   Bob: $50 → $80                                         │   │
│  │                                                          │   │
│  │ Simple subtraction/addition                                │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
│  Bitcoin UTXO Model:                                            │
│  ────────────────────                                           │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Think of UTXOs as "coins" or "bills" you own           │   │
│  │                                                          │   │
│  │ Alice owns:                                              │   │
│  │   • UTXO A: 40 BTC (from previous transaction)         │   │
│  │   • UTXO B: 60 BTC (from another transaction)          │   │
│  │                                                          │   │
│  │ Alice wants to send 50 BTC to Bob:                     │   │
│  │                                                          │   │
│  │   Inputs (what Alice spends):                          │   │
│  │     • UTXO A: 40 BTC                                     │   │
│  │     • UTXO B: 60 BTC                                     │   │
│  │     Total input: 100 BTC                                 │   │
│  │                                                          │   │
│  │   Outputs (what gets created):                         │   │
│  │     • Output 1: 50 BTC → Bob (payment)                  │   │
│  │     • Output 2: 49.999 BTC → Alice (change)             │   │
│  │     • Output 3: 0.001 BTC → Miner (fee)                 │   │
│  │     Total output: 100 BTC                                │   │
│  │                                                          │   │
│  │   UTXO A and B are now SPENT (destroyed)                │   │
│  │   New UTXOs are created for Bob and Alice               │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
│  Key Rules:                                                     │
│  ───────────                                                    │
│  • UTXOs can only be spent ONCE (preventing double-spend)      │
│  • Must spend entire UTXO (cannot spend partial)               │
│  • Change comes back to you as a new UTXO                      │
│  • Transaction fees = Inputs - Outputs                         │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Visual UTXO Flow:**

```
UTXO Lifecycle:

Transaction 1 (Alice receives from Coinbase/mining):
┌─────────────────────────────────────────────────────────────────┐
│  Inputs: (none - coinbase transaction)                        │
│                                                                 │
│  Outputs:                                                       │
│  ┌─────────────┐                                               │
│  │ UTXO #1     │                                               │
│  │ Value: 50   │───▶ Alice owns this                           │
│  │ Owner: Alice│                                               │
│  └─────────────┘                                               │
│                                                                 │
│  Alice's Wallet: [UTXO #1: 50 BTC]                             │
└─────────────────────────────────────────────────────────────────┘
                              │
                              ▼
Transaction 2 (Alice sends 30 BTC to Bob):
┌─────────────────────────────────────────────────────────────────┐
│  Inputs:                                                        │
│  ┌─────────────┐                                                │
│  │ UTXO #1     │───▶ Spent (destroyed)                          │
│  │ Value: 50   │                                                │
│  └─────────────┘                                                │
│                                                                 │
│  Outputs:                                                       │
│  ┌─────────────┐    ┌─────────────┐                           │
│  │ UTXO #2     │    │ UTXO #3      │                           │
│  │ Value: 30   │───▶│ Value: 19.999│───▶ Alice's change        │
│  │ Owner: Bob  │    │ Owner: Alice │                           │
│  └─────────────┘    └─────────────┘                           │
│         │                                                       │
│         └──────────────────────────────────▶ Bob receives      │
│                                                                 │
│  Alice's Wallet: [UTXO #3: 19.999 BTC]                         │
│  Bob's Wallet:   [UTXO #2: 30 BTC]                             │
└─────────────────────────────────────────────────────────────────┘
                              │
                              ▼
Transaction 3 (Bob sends 10 BTC to Carol):
┌─────────────────────────────────────────────────────────────────┐
│  Inputs:                                                        │
│  ┌─────────────┐                                                │
│  │ UTXO #2     │───▶ Spent (destroyed)                          │
│  │ Value: 30   │                                                │
│  └─────────────┘                                                │
│                                                                 │
│  Outputs:                                                       │
│  ┌─────────────┐    ┌─────────────┐                           │
│  │ UTXO #4     │    │ UTXO #5      │                           │
│  │ Value: 10   │───▶│ Value: 19.999│───▶ Bob's change          │
│  │ Owner: Carol│    │ Owner: Bob   │                           │
│  └─────────────┘    └─────────────┘                           │
│         │                                                       │
│         └──────────────────────────────────▶ Carol receives      │
│                                                                 │
│  Alice's Wallet: [UTXO #3: 19.999 BTC]                         │
│  Bob's Wallet:   [UTXO #5: 19.999 BTC]                         │
│  Carol's Wallet: [UTXO #4: 10 BTC]                             │
└─────────────────────────────────────────────────────────────────┘
```

**Code Example: UTXO Model Implementation**

```javascript
// UTXO Model Implementation
const crypto = require('crypto');

/**
 * Create a SHA-256 hash
 */
function sha256(data) {
    return crypto.createHash('sha256').update(data).digest('hex');
}

/**
 * UTXO Class - Represents an unspent transaction output
 */
class UTXO {
    constructor(txId, outputIndex, address, amount) {
        this.txId = txId;              // Transaction ID that created this UTXO
        this.outputIndex = outputIndex; // Index in the output array
        this.address = address;        // Owner's address (public key hash)
        this.amount = amount;          // Amount of cryptocurrency
        this.spent = false;            // Whether this UTXO has been spent
    }
    
    /**
     * Create a unique identifier for this UTXO
     */
    getId() {
        return `${this.txId}:${this.outputIndex}`;
    }
    
    /**
     * Mark this UTXO as spent
     */
    spend() {
        this.spent = true;
    }
    
    display() {
        return {
            id: this.getId(),
            address: this.address.substring(0, 20) + '...',
            amount: this.amount,
            status: this.spent ? 'SPENT' : 'UNSPENT'
        };
    }
}

/**
 * Transaction class for UTXO model
 */
class UTXOTransaction {
    constructor(inputs = [], outputs = []) {
        this.inputs = inputs;      // Array of {txId, outputIndex, signature}
        this.outputs = outputs;      // Array of {address, amount}
        this.timestamp = Date.now();
        this.txId = this.calculateHash(); // Transaction ID
    }
    
    /**
     * Calculate transaction hash (TXID)
     */
    calculateHash() {
        const txData = JSON.stringify({
            inputs: this.inputs,
            outputs: this.outputs,
            timestamp: this.timestamp
        });
        return sha256(txData);
    }
    
    /**
     * Add an input (reference to UTXO being spent)
     */
    addInput(utxoId, signature) {
        const [txId, outputIndex] = utxoId.split(':');
        this.inputs.push({
            txId,
            outputIndex: parseInt(outputIndex),
            signature
        });
    }
    
    /**
     * Add an output (new UTXO being created)
     */
    addOutput(address, amount) {
        this.outputs.push({ address, amount });
    }
    
    /**
     * Validate the transaction
     */
    validate(utxoPool) {
        // Check that inputs exist and are unspent
        let inputSum = 0;
        for (const input of this.inputs) {
            const utxoId = `${input.txId}:${input.outputIndex}`;
            const utxo = utxoPool[utxoId];
            
            if (!utxo) {
                return { valid: false, error: `UTXO ${utxoId} not found` };
            }
            
            if (utxo.spent) {
                return { valid: false, error: `UTXO ${utxoId} already spent` };
            }
            
            inputSum += utxo.amount;
        }
        
        // Check that outputs don't exceed inputs
        const outputSum = this.outputs.reduce((sum, out) => sum + out.amount, 0);
        
        if (outputSum > inputSum) {
            return { 
                valid: false, 
                error: `Output sum (${outputSum}) exceeds input sum (${inputSum})` 
            };
        }
        
        // Calculate fee
        const fee = inputSum - outputSum;
        
        return { valid: true, fee };
    }
    
    display() {
        return {
            txId: this.txId.substring(0, 16) + '...',
            inputs: this.inputs.length,
            outputs: this.outputs.map((o, i) => ({
                index: i,
                address: o.address.substring(0, 20) + '...',
                amount: o.amount
            })),
            timestamp: new Date(this.timestamp).toISOString()
        };
    }
}

/**
 * UTXO Pool - Manages all unspent transaction outputs
 */
class UTXOPool {
    constructor() {
        this.utxos = {};  // Map of "txId:index" -> UTXO
    }
    
    /**
     * Add a new UTXO to the pool
     */
    addUTXO(utxo) {
        this.utxos[utxo.getId()] = utxo;
    }
    
    /**
     * Get UTXO by ID
     */
    getUTXO(id) {
        return this.utxos[id];
    }
    
    /**
     * Get all UTXOs for a specific address
     */
    getUTXOsForAddress(address) {
        return Object.values(this.utxos).filter(utxo => 
            utxo.address === address && !utxo.spent
        );
    }
    
    /**
     * Process a transaction (spend inputs, create outputs)
     */
    processTransaction(tx) {
        // Validate
        const validation = tx.validate(this.utxos);
        if (!validation.valid) {
            throw new Error(validation.error);
        }
        
        // Mark inputs as spent
        for (const input of tx.inputs) {
            const utxoId = `${input.txId}:${input.outputIndex}`;
            this.utxos[utxoId].spend();
        }
        
        // Create new UTXOs for outputs
        tx.outputs.forEach((output, index) => {
            const utxo = new UTXO(tx.txId, index, output.address, output.amount);
            this.addUTXO(utxo);
        });
        
        return validation.fee;
    }
    
    /**
     * Get total supply (sum of all unspent UTXOs)
     */
    getTotalSupply() {
        return Object.values(this.utxos)
            .filter(utxo => !utxo.spent)
            .reduce((sum, utxo) => sum + utxo.amount, 0);
    }
    
    display() {
        const unspent = Object.values(this.utxos).filter(u => !u.spent);
        console.log(`\nUTXO Pool Status:`);
        console.log(`  Total UTXOs: ${Object.keys(this.utxos).length}`);
        console.log(`  Unspent UTXOs: ${unspent.length}`);
        console.log(`  Total Supply: ${this.getTotalSupply()}`);
        console.log(`\nUnspent UTXOs:`);
        unspent.forEach(utxo => {
            console.log(`  - ${utxo.getId()}: ${utxo.amount} BTC`);
        });
    }
}

// ==================== DEMONSTRATION ====================

console.log('=== UTXO MODEL DEMONSTRATION ===\n');

const utxoPool = new UTXOPool();

// Create addresses (simplified - normally derived from public keys)
const aliceAddress = '1AliceAddress' + 'x'.repeat(23);
const bobAddress = '1BobAddress' + 'x'.repeat(25);
const carolAddress = '1CarolAddress' + 'x'.repeat(23);

// Create initial UTXO for Alice (like a coinbase transaction)
console.log('1. Creating initial UTXO for Alice (mining reward):');
const genesisUTXO = new UTXO('genesis', 0, aliceAddress, 50);
utxoPool.addUTXO(genesisUTXO);
console.log('   Alice receives: 50 BTC');
utxoPool.display();

// Alice sends 30 BTC to Bob, gets 19.999 change, 0.001 fee
console.log('\n2. Alice sends 30 BTC to Bob:');
const tx1 = new UTXOTransaction();
tx1.addInput('genesis:0', 'signature_alice');
tx1.addOutput(bobAddress, 30);
tx1.addOutput(aliceAddress, 19.999); // Change

console.log('   Transaction created:', tx1.display());

const fee1 = utxoPool.processTransaction(tx1);
console.log(`   Fee paid: ${fee1} BTC`);
utxoPool.display();

// Bob sends 10 BTC to Carol
console.log('\n3. Bob sends 10 BTC to Carol:');
const tx2 = new UTXOTransaction();
tx2.addInput(tx1.txId + ':0', 'signature_bob'); // Bob's output from tx1
tx2.addOutput(carolAddress, 10);
tx2.addOutput(bobAddress, 19.998); // Change (minus 0.002 fee)

console.log('   Transaction created:', tx2.display());

const fee2 = utxoPool.processTransaction(tx2);
console.log(`   Fee paid: ${fee2} BTC`);
utxoPool.display();

// Try to spend already spent UTXO (should fail)
console.log('\n4. Attempting double-spend (should fail):');
const tx3 = new UTXOTransaction();
tx3.addInput('genesis:0', 'signature_eve'); // Try to spend genesis again

try {
    utxoPool.processTransaction(tx3);
    console.log('   ERROR: Transaction should have failed!');
} catch (error) {
    console.log(`   ✓ Correctly rejected: ${error.message}`);
}

console.log('\nFinal Balances:');
console.log(`  Alice: ${utxoPool.getUTXOsForAddress(aliceAddress).reduce((s, u) => s + u.amount, 0)} BTC`);
console.log(`  Bob: ${utxoPool.getUTXOsForAddress(bobAddress).reduce((s, u) => s + u.amount, 0)} BTC`);
console.log(`  Carol: ${utxoPool.getUTXOsForAddress(carolAddress).reduce((s, u) => s + u.amount, 0)} BTC`);

/*
EXPECTED OUTPUT:

=== UTXO MODEL DEMONSTRATION ===

1. Creating initial UTXO for Alice (mining reward):
   Alice receives: 50 BTC

UTXO Pool Status:
  Total UTXOs: 1
  Unspent UTXOs: 1
  Total Supply: 50

Unspent UTXOs:
  - genesis:0: 50 BTC

2. Alice sends 30 BTC to Bob:
   Transaction created: {
     txId: 'a3b2c1d0...',
     inputs: 1,
     outputs: [ [Object], [Object] ],
     timestamp: '2024-01-15T10:30:00.000Z'
   }
   Fee paid: 0.001 BTC

UTXO Pool Status:
  Total UTXOs: 3
  Unspent UTXOs: 2
  Total Supply: 49.999

Unspent UTXOs:
  - a3b2c1d0...:1: 19.999 BTC (Alice's change)
  - a3b2c1d0...:0: 30 BTC (Bob's payment)

... (continues)
*/
```

**Explanation of UTXO Code:**

1. **`UTXO` class**: Represents an unspent output with txId, index, address, and amount. Tracks whether it's been spent.

2. **`UTXOTransaction` class**: 
   - Contains inputs (references to existing UTXOs) and outputs (new UTXOs to create)
   - `calculateHash()` creates the transaction ID
   - `validate()` checks that inputs exist, aren't already spent, and that outputs don't exceed inputs

3. **`UTXOPool` class**:
   - Manages all UTXOs in the system
   - `processTransaction()` validates, marks inputs spent, and creates new UTXOs
   - Prevents double-spending by checking `spent` flag

4. **Fee calculation**: Difference between input and output amounts goes to miners as fee.

### **3.3.3 Account-Based Model**

Ethereum uses an account-based model instead of UTXOs. This is more intuitive and similar to traditional banking.

```
Account-Based Model (Ethereum):

┌─────────────────────────────────────────────────────────────────┐
│                    ACCOUNT MODEL                                │
│                                                                 │
│  Global State:                                                  │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Address → Account Data                                  │   │
│  │                                                         │   │
│  │ 0x742d...35Cc → {                                       │   │
│  │   balance: 10.5 ETH,                                    │   │
│  │   nonce: 5,                                             │   │
│  │   code: null,    // null = EOA (Externally Owned)      │   │
│  │   storage: {}    // Smart contract storage              │   │
│  │ }                                                       │   │
│  │                                                         │   │
│  │ 0x5Ea1...f150 → {                                       │   │
│  │   balance: 2.3 ETH,                                     │   │
│  │   nonce: 12,                                            │   │
│  │   code: <bytecode>,  // Smart contract                   │   │
│  │   storage: {0: 100, 1: "data"}                         │   │
│  │ }                                                       │   │
│  │                                                         │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
│  Transaction Structure:                                         │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ {                                                       │   │
│  │   nonce: 5,        // Transaction counter              │   │
│  │   gasPrice: 20,    // Fee per gas unit                 │   │
│  │   gasLimit: 21000, // Max gas willing to spend          │   │
│  │   to: "0x5Ea1...", // Recipient (null for contract)     │   │
│  │   value: 1.5,      // ETH to transfer                  │   │
│  │   data: "0x",      // Contract call data               │   │
│  │   v, r, s: ...     // Signature                        │   │
│  │ }                                                       │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
│  How Transfer Works:                                            │
│  ───────────────────                                            │
│  Before:                                                        │
│    Alice: 10 ETH, nonce: 5                                       │
│    Bob: 2 ETH                                                  │
│                                                                 │
│  Transaction: Alice sends 1 ETH to Bob, nonce=5                │
│                                                                 │
│  Validation:                                                    │
│    ✓ Alice's nonce matches (prevents replay)                   │
│    ✓ Alice has enough balance (10 >= 1 + fee)                  │
│    ✓ Signature is valid                                        │
│                                                                 │
│  After:                                                         │
│    Alice: 9 ETH (minus fee), nonce: 6                          │
│    Bob: 3 ETH                                                  │
│                                                                 │
│  Key Differences from UTXO:                                     │
│  ──────────────────────────                                     │
│  • No "change" outputs needed                                  │
│  • Nonce prevents replay attacks instead of UTXO uniqueness     │
│  • Can send partial amounts easily                              │
│  • More intuitive for developers                               │
│  • Easier to query balance (single value vs sum of UTXOs)     │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Code Example: Account Model Implementation**

```javascript
// Account-Based Model (Ethereum-style)
const crypto = require('crypto');

/**
 * Account class
 */
class Account {
    constructor(address, balance = 0) {
        this.address = address;
        this.balance = balance;
        this.nonce = 0;              // Transaction counter
        this.code = null;            // Smart contract code (null for EOA)
        this.storage = {};           // Contract storage
    }
    
    /**
     * Increment nonce (called after each transaction)
     */
    incrementNonce() {
        this.nonce++;
    }
    
    /**
     * Add to balance
     */
    credit(amount) {
        this.balance += amount;
    }
    
    /**
     * Subtract from balance
     */
    debit(amount) {
        if (this.balance < amount) {
            throw new Error('Insufficient balance');
        }
        this.balance -= amount;
    }
    
    display() {
        return {
            address: this.address.substring(0, 20) + '...',
            balance: this.balance,
            nonce: this.nonce,
            type: this.code ? 'Contract' : 'EOA'
        };
    }
}

/**
 * Account-based transaction
 */
class AccountTransaction {
    constructor(from, to, value, gasPrice, gasLimit, data = '', nonce) {
        this.from = from;
        this.to = to;
        this.value = value;
        this.gasPrice = gasPrice;
        this.gasLimit = gasLimit;
        this.data = data;
        this.nonce = nonce;  // Must match sender's current nonce
        this.timestamp = Date.now();
        this.hash = this.calculateHash();
    }
    
    calculateHash() {
        const txData = JSON.stringify({
            from: this.from,
            to: this.to,
            value: this.value,
            gasPrice: this.gasPrice,
            gasLimit: this.gasLimit,
            data: this.data,
            nonce: this.nonce,
            timestamp: this.timestamp
        });
        return crypto.createHash('sha256').update(txData).digest('hex');
    }
    
    /**
     * Calculate maximum cost (value + max gas fee)
     */
    getMaxCost() {
        return this.value + (this.gasPrice * this.gasLimit);
    }
}

/**
 * State manager for account-based blockchain
 */
class StateManager {
    constructor() {
        this.accounts = {};  // address -> Account
        this.transactions = []; // All processed transactions
    }
    
    /**
     * Create or get account
     */
    getOrCreateAccount(address) {
        if (!this.accounts[address]) {
            this.accounts[address] = new Account(address);
        }
        return this.accounts[address];
    }
    
    /**
     * Get account balance
     */
    getBalance(address) {
        const account = this.accounts[address];
        return account ? account.balance : 0;
    }
    
    /**
     * Get account nonce
     */
    getNonce(address) {
        const account = this.accounts[address];
        return account ? account.nonce : 0;
    }
    
    /**
     * Process a transaction (state transition)
     */
    processTransaction(tx) {
        // Get sender account
        const sender = this.getOrCreateAccount(tx.from);
        
        // Validation 1: Check nonce (prevents replay attacks)
        if (tx.nonce !== sender.nonce) {
            throw new Error(
                `Invalid nonce. Expected: ${sender.nonce}, Got: ${tx.nonce}`
            );
        }
        
        // Validation 2: Check sufficient balance for value + gas
        const maxCost = tx.getMaxCost();
        if (sender.balance < maxCost) {
            throw new Error(
                `Insufficient balance. Have: ${sender.balance}, Need: ${maxCost}`
            );
        }
        
        // Validation 3: Gas limit must be at least intrinsic gas
        const intrinsicGas = 21000; // Basic transaction cost
        if (tx.gasLimit < intrinsicGas) {
            throw new Error(`Gas limit too low. Minimum: ${intrinsicGas}`);
        }
        
        // Execute transaction
        // 1. Deduct value from sender
        sender.debit(tx.value);
        
        // 2. Credit value to recipient
        const recipient = this.getOrCreateAccount(tx.to);
        recipient.credit(tx.value);
        
        // 3. Charge gas fee (simplified - normally based on actual gas used)
        const gasUsed = intrinsicGas;
        const gasFee = gasUsed * tx.gasPrice;
        sender.debit(gasFee);
        
        // 4. Increment sender nonce
        sender.incrementNonce();
        
        // 5. Record transaction
        this.transactions.push({
            hash: tx.hash,
            from: tx.from,
            to: tx.to,
            value: tx.value,
            gasUsed: gasUsed,
            gasFee: gasFee,
            timestamp: tx.timestamp
        });
        
        return {
            success: true,
            gasUsed: gasUsed,
            gasFee: gasFee,
            senderNewBalance: sender.balance,
            recipientNewBalance: recipient.balance
        };
    }
    
    /**
     * Display current state
     */
    displayState() {
        console.log('\n=== CURRENT STATE ===');
        console.log(`Total Accounts: ${Object.keys(this.accounts).length}`);
        console.log(`Total Transactions: ${this.transactions.length}`);
        console.log('\nAccount Balances:');
        Object.values(this.accounts).forEach(acc => {
            console.log(`  ${acc.address}: ${acc.balance} ETH (nonce: ${acc.nonce})`);
        });
    }
}

// ==================== DEMONSTRATION ====================

console.log('=== ACCOUNT-BASED MODEL DEMONSTRATION ===\n');

const state = new StateManager();

// Create accounts
const alice = '0x742d35Cc6634C0532925a3b844Bc9e7595f3bAd';
const bob = '0x5Ea1bb242326044699C3d81341c5f535d5Af1504';

// Initialize Alice with some ETH (like from mining or genesis)
console.log('1. Initializing Alice with 10 ETH:');
state.getOrCreateAccount(alice).credit(10);
state.displayState();

// Alice sends 2 ETH to Bob
console.log('\n2. Alice sends 2 ETH to Bob:');
const tx1 = new AccountTransaction(
    alice,      // from
    bob,        // to
    2,          // value (ETH)
    0.00000002, // gasPrice (20 gwei)
    21000,      // gasLimit
    '',         // data
    0           // nonce (Alice's first transaction)
);

console.log('   Transaction:', {
    from: tx1.from.substring(0, 10) + '...',
    to: tx1.to.substring(0, 10) + '...',
    value: tx1.value + ' ETH',
    nonce: tx1.nonce
});

const result1 = state.processTransaction(tx1);
console.log('   Result:', result1);
state.displayState();

// Alice sends another 1 ETH to Bob (nonce should be 1 now)
console.log('\n3. Alice sends another 1 ETH to Bob:');
const tx2 = new AccountTransaction(
    alice,
    bob,
    1,
    0.00000002,
    21000,
    '',
    1  // Nonce incremented!
);

const result2 = state.processTransaction(tx2);
console.log('   Result:', result2);
state.displayState();

// Try replay attack with old nonce (should fail)
console.log('\n4. Attempting replay attack with old nonce (should fail):');
const tx3 = new AccountTransaction(
    alice,
    bob,
    5,
    0.00000002,
    21000,
    '',
    0  // Old nonce!
);

try {
    state.processTransaction(tx3);
    console.log('   ERROR: Should have failed!');
} catch (error) {
    console.log(`   ✓ Correctly rejected: ${error.message}`);
}

// Try with insufficient balance (should fail)
console.log('\n5. Attempting to send more than balance (should fail):');
const tx4 = new AccountTransaction(
    alice,
    bob,
    100,  // Way more than Alice has
    0.00000002,
    21000,
    '',
    2   // Correct nonce
);

try {
    state.processTransaction(tx4);
    console.log('   ERROR: Should have failed!');
} catch (error) {
    console.log(`   ✓ Correctly rejected: ${error.message}`);
}

console.log('\nFinal State:');
state.displayState();

/*
EXPECTED OUTPUT:

=== ACCOUNT-BASED MODEL DEMONSTRATION ===

1. Initializing Alice with 10 ETH:

=== CURRENT STATE ===
Total Accounts: 1
Total Transactions: 0

Account Balances:
  0x742d35Cc6634C0532925a...: 10 ETH (nonce: 0)

2. Alice sends 2 ETH to Bob:
   Transaction: { from: '0x742d35Cc...', to: '0x5Ea1bb24...', value: '2 ETH', nonce: 0 }
   Result: { success: true, gasUsed: 21000, gasFee: 0.00042, senderNewBalance: 7.99958, recipientNewBalance: 2 }

=== CURRENT STATE ===
Total Accounts: 2
Total Transactions: 1

Account Balances:
  0x742d35Cc6634C0532925a...: 7.99958 ETH (nonce: 1)
  0x5Ea1bb242326044699C3d8...: 2 ETH (nonce: 0)

... (continues)
*/
```

### **3.3.4 Transaction Fees and Gas**

Transaction fees serve two purposes: preventing spam and compensating miners/validators.

```
Transaction Fee Mechanics:

┌─────────────────────────────────────────────────────────────────┐
│                    FEE CALCULATION                              │
│                                                                 │
│  Bitcoin (UTXO Model):                                          │
│  ─────────────────────                                          │
│  Fee = Sum(Inputs) - Sum(Outputs)                              │
│                                                                 │
│  Example:                                                       │
│  Inputs:  1.0 BTC                                               │
│  Outputs: 0.7 BTC (to recipient) + 0.299 BTC (change)          │
│  Fee:     0.001 BTC (implicit - remainder)                     │
│                                                                 │
│  Fee rate: satoshis per byte (sat/vB)                          │
│  Typical: 1-20 sat/vB depending on network congestion          │
│                                                                 │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  Ethereum (Account Model):                                      │
│  ─────────────────────────                                      │
│  Fee = Gas Used × Gas Price                                    │
│                                                                 │
│  Gas: Unit of computational work                               │
│  • Basic transfer: 21,000 gas                                  │
│  • Complex contract: 100,000+ gas                              │
│                                                                 │
│  Gas Price: Price per unit of gas (in gwei)                    │
│  1 gwei = 0.000000001 ETH                                      │
│                                                                 │
│  Example:                                                       │
│  Gas Used:     21,000                                           │
│  Gas Price:    20 gwei                                          │
│  Fee:          21,000 × 20 = 420,000 gwei = 0.00042 ETH        │
│                                                                 │
│  EIP-1559 (London Upgrade):                                     │
│  Fee = Gas × (Base Fee + Priority Fee)                         │
│  • Base Fee: Burned (removed from circulation)                  │
│  • Priority Fee: Goes to miner/validator                        │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **3.3.5 Nonce and Transaction Ordering**

The nonce is critical for preventing replay attacks and ensuring transaction order.

```
Nonce (Number Used Once) in Ethereum:

┌─────────────────────────────────────────────────────────────────┐
│                    NONCE PURPOSES                               │
│                                                                 │
│  1. PREVENTING REPLAY ATTACKS                                   │
│     ─────────────────────────                                   │
│     Without nonce:                                              │
│     • Alice sends 1 ETH to Bob (valid)                         │
│     • Eve captures the transaction                             │
│     • Eve rebroadcasts it                                      │
│     • Alice loses another 1 ETH (replay!)                      │
│                                                                 │
│     With nonce:                                                 │
│     • Alice sends 1 ETH with nonce=5 (valid)                   │
│     • Eve captures and rebroadcasts                            │
│     • Network rejects: nonce 5 already used!                   │
│     • Alice is protected                                        │
│                                                                 │
│  2. ENSURING TRANSACTION ORDER                                  │
│     ──────────────────────────                                  │
│     Transactions must be processed in nonce order:             │
│     • Nonce 5 must be mined before nonce 6                     │
│     • If nonce 6 arrives first, it's held in mempool           │
│     • Once nonce 5 is mined, nonce 6 becomes valid             │
│                                                                 │
│  3. REPLACEMENT TRANSACTIONS                                    │
│     ─────────────────────────                                   │
│     Same nonce + higher gas price = replacement                │
│     • Alice sends tx with nonce=5, gas=20 gwei                 │
│     • Network congested, tx stuck                              │
│     • Alice sends new tx with nonce=5, gas=50 gwei             │
│     • Miners prefer higher fee, replace old tx                 │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## **3.4 Memory Pool (Mempool)**

### **3.4.1 What is the Mempool?**

The mempool (memory pool) is a temporary holding area for unconfirmed transactions.

```
Mempool Concept:

┌─────────────────────────────────────────────────────────────────┐
│                    TRANSACTION LIFECYCLE                        │
│                                                                 │
│  Step 1: Creation                                               │
│  User creates and signs transaction                            │
│                              │                                  │
│                              ▼                                  │
│  Step 2: Submission                                             │
│  Broadcast to network                                          │
│                              │                                  │
│                              ▼                                  │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │                    MEMPOOL                               │   │
│  │              (Waiting Area)                              │   │
│  │                                                         │   │
│  │  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐     │   │
│  │  │ Tx A: 20    │  │ Tx B: 15    │  │ Tx C: 30    │     │   │
│  │  │ gwei fee    │  │ gwei fee    │  │ gwei fee    │     │   │
│  │  │             │  │             │  │             │     │   │
│  │  │ [Waiting]   │  │ [Waiting]   │  │ [Waiting]   │     │   │
│  │  └─────────────┘  └─────────────┘  └─────────────┘     │   │
│  │                                                         │   │
│  │  Each node has its own mempool view                     │   │
│  │  Size limited (e.g., 300 MB in Bitcoin)                │   │
│  │  Eviction policy: Lowest fee transactions dropped first │   │
│  │                                                         │   │
│  └─────────────────────────────────────────────────────────┘   │
│                              │                                  │
│                              ▼                                  │
│  Step 3: Mining/Validation                                      │
│  Miner selects transactions from mempool                       │
│  (Usually highest fee first)                                   │
│                              │                                  │
│                              ▼                                  │
│  Step 4: Confirmation                                           │
│  Block mined, transactions confirmed                           │
│  Removed from all mempools                                     │
│                              │                                  │
│                              ▼                                  │
│  Step 5: Finality                                               │
│  More blocks added on top                                      │
│  Becomes irreversible                                          │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **3.4.2 Transaction Propagation**

When a transaction is broadcast, it spreads through the network using a gossip protocol.

```
Gossip Protocol:

┌─────────────────────────────────────────────────────────────────┐
│              TRANSACTION PROPAGATION                            │
│                                                                 │
│  Alice broadcasts transaction:                                  │
│                                                                 │
│       ┌─────────┐                                               │
│       │ Alice's │                                               │
│       │  Node   │                                               │
│       └────┬────┘                                               │
│            │ Broadcast to peers                                  │
│            │                                                    │
│     ┌──────┼──────┐                                             │
│     ▼      ▼      ▼                                             │
│  ┌────┐  ┌────┐  ┌────┐                                        │
│  │Node│  │Node│  │Node│  ◀── Each validates then forwards      │
│  │ 1  │  │ 2  │  │ 3  │                                         │
│  └─┬──┘  └─┬──┘  └─┬──┘                                         │
│    │       │       │                                            │
│    ▼       ▼       ▼                                            │
│  Spread to their peers...                                       │
│                                                                 │
│  Within seconds, transaction reaches entire network             │
│                                                                 │
│  Validation before forwarding:                                  │
│  • Signature valid?                                             │
│  • Nonce correct?                                               │
│  • Sufficient balance?                                          │
│  • Not already in mempool?                                      │
│  • Meets minimum fee?                                           │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **3.4.3 Fee Market Dynamics**

The mempool creates a fee market where users compete for block space.

```
Fee Market Visualization:

┌─────────────────────────────────────────────────────────────────┐
│                    FEE MARKET                                   │
│                                                                 │
│  Block Space: Limited (~1-2 MB per 10 min in Bitcoin)          │
│  Demand: Variable (more transactions than space)               │
│                                                                 │
│  Visual representation of mempool:                              │
│                                                                 │
│  Block Limit ───────────────────────────────┐                   │
│                                             │                   │
│  High Fee   ████████████████  50+ sat/vB   │  ← Included fast  │
│             ████████████████               │                   │
│  Medium Fee ██████████       20-50 sat/vB  │  ← Included soon  │
│             ████████                       │                   │
│  Low Fee    ████             1-20 sat/vB   │  ← May wait hours │
│             ██                             │                   │
│  Dust       █                  <1 sat/vB   │  ← Likely stuck   │
│             │                              │                   │
│  Minimum    │                              │                   │
│  Relay Fee ─┘                              │                   │
│                                             │                   │
│  Strategy: Pay higher fee = faster confirmation                 │
│                                                                 │
│  Fee Estimation Algorithms:                                     │
│  • Look at mempool composition                                  │
│  • Estimate based on recent blocks                              │
│  • Predict required fee for target confirmation time           │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## **3.5 Building a Simple Blockchain from Scratch**

Now let's put everything together to build a working blockchain.

### **3.5.1 Project Setup**

First, let's create the basic structure:

```javascript
// simple-blockchain.js
// A complete, working blockchain implementation

const crypto = require('crypto');

// Utility function for SHA-256
function sha256(data) {
    return crypto.createHash('sha256').update(data).digest('hex');
}

// Utility for creating Merkle root (simplified)
function createMerkleRoot(transactions) {
    if (transactions.length === 0) {
        return sha256('empty');
    }
    if (transactions.length === 1) {
        return transactions[0].hash;
    }
    
    let hashes = transactions.map(tx => tx.hash);
    
    while (hashes.length > 1) {
        const nextLevel = [];
        for (let i = 0; i < hashes.length; i += 2) {
            if (i + 1 === hashes.length) {
                nextLevel.push(sha256(hashes[i] + hashes[i]));
            } else {
                nextLevel.push(sha256(hashes[i] + hashes[i + 1]));
            }
        }
        hashes = nextLevel;
    }
    
    return hashes[0];
}
```

### **3.5.2 Creating the Block Class**

```javascript
/**
 * Block Class - Represents a single block in the blockchain
 */
class Block {
    constructor(index, previousHash, transactions, difficulty = 2) {
        this.index = index;
        this.previousHash = previousHash;
        this.transactions = transactions;
        this.timestamp = Date.now();
        this.difficulty = difficulty;
        this.nonce = 0;
        this.merkleRoot = createMerkleRoot(transactions);
        this.hash = this.calculateHash();
    }
    
    /**
     * Calculate block hash from header fields
     */
    calculateHash() {
        const header = this.index + this.previousHash + 
                      this.merkleRoot + this.timestamp + 
                      this.difficulty + this.nonce;
        return sha256(header);
    }
    
    /**
     * Proof of Work mining
     * Find a hash that starts with '0' repeated difficulty times
     */
    mineBlock() {
        const target = '0'.repeat(this.difficulty);
        
        console.log(`Mining block ${this.index}...`);
        const startTime = Date.now();
        
        while (this.hash.substring(0, this.difficulty) !== target) {
            this.nonce++;
            this.hash = this.calculateHash();
        }
        
        const duration = (Date.now() - startTime) / 1000;
        console.log(`Block mined: ${this.hash}`);
        console.log(`Nonce: ${this.nonce}`);
        console.log(`Time: ${duration}s\n`);
    }
    
    /**
     * Check if all transactions in block are valid
     */
    hasValidTransactions(utxoPool) {
        for (const tx of this.transactions) {
            if (!tx.isValid(utxoPool)) {
                return false;
            }
        }
        return true;
    }
    
    display() {
        return {
            index: this.index,
            hash: this.hash.substring(0, 20) + '...',
            previousHash: this.previousHash.substring(0, 20) + '...',
            merkleRoot: this.merkleRoot.substring(0, 20) + '...',
            timestamp: new Date(this.timestamp).toISOString(),
            transactions: this.transactions.length,
            nonce: this.nonce,
            difficulty: this.difficulty
        };
    }
}
```

### **3.5.3 Implementing the Blockchain Class**

```javascript
/**
 * Blockchain Class - Manages the chain of blocks
 */
class Blockchain {
    constructor(difficulty = 2) {
        this.chain = [this.createGenesisBlock()];
        this.difficulty = difficulty;
        this.pendingTransactions = [];
        this.miningReward = 100;
        this.utxoPool = new Map(); // Track unspent outputs
    }
    
    /**
     * Create the first block
     */
    createGenesisBlock() {
        const genesisCoinbase = new Transaction(
            null, 
            'genesis-address', 
            1000000, 
            'coinbase'
        );
        return new Block(0, '0'.repeat(64), [genesisCoinbase], this.difficulty);
    }
    
    /**
     * Get the latest block
     */
    getLatestBlock() {
        return this.chain[this.chain.length - 1];
    }
    
    /**
     * Add a new transaction to pending list
     */
    addTransaction(transaction) {
        // Validate transaction
        if (!transaction.from || !transaction.to) {
            throw new Error('Transaction must include from and to address');
        }
        
        if (!transaction.isValid(this.utxoPool)) {
            throw new Error('Cannot add invalid transaction to chain');
        }
        
        this.pendingTransactions.push(transaction);
        console.log(`Transaction added to pending: ${transaction.hash.substring(0, 16)}...`);
    }
    
    /**
     * Mine pending transactions
     */
    minePendingTransactions(miningRewardAddress) {
        // Create coinbase transaction (mining reward)
        const rewardTx = new Transaction(
            null, 
            miningRewardAddress, 
            this.miningReward,
            'coinbase'
        );
        
        // Include reward in block
        const blockTransactions = [rewardTx, ...this.pendingTransactions];
        
        // Create new block
        const block = new Block(
            this.chain.length,
            this.getLatestBlock().hash,
            blockTransactions,
            this.difficulty
        );
        
        // Mine the block (find valid nonce)
        block.mineBlock();
        
        // Add to chain
        this.chain.push(block);
        
        // Update UTXO pool
        this.updateUTXOPool(block);
        
        // Reset pending transactions
        this.pendingTransactions = [];
        
        console.log(`Block successfully added to chain!\n`);
        return block;
    }
    
    /**
     * Update UTXO pool based on block transactions
     */
    updateUTXOPool(block) {
        for (const tx of block.transactions) {
            // Remove spent inputs (skip for coinbase)
            if (tx.from) {
                const utxoKey = `${tx.fromTxId}:${tx.fromOutputIndex}`;
                this.utxoPool.delete(utxoKey);
            }
            
            // Add new outputs
            const utxoKey = `${tx.hash}:0`; // Simplified - assume 1 output
            this.utxoPool.set(utxoKey, {
                address: tx.to,
                amount: tx.amount
            });
        }
    }
    
    /**
     * Validate entire chain integrity
     */
    isChainValid() {
        for (let i = 1; i < this.chain.length; i++) {
            const currentBlock = this.chain[i];
            const previousBlock = this.chain[i - 1];
            
            // Check if block hash is valid
            if (currentBlock.hash !== currentBlock.calculateHash()) {
                console.error(`Block ${i} has invalid hash`);
                return false;
            }
            
            // Check if previous hash matches
            if (currentBlock.previousHash !== previousBlock.hash) {
                console.error(`Block ${i} has invalid previous hash`);
                return false;
            }
            
            // Check if proof of work is valid
            const target = '0'.repeat(currentBlock.difficulty);
            if (currentBlock.hash.substring(0, currentBlock.difficulty) !== target) {
                console.error(`Block ${i} has invalid proof of work`);
                return false;
            }
        }
        
        return true;
    }
    
    /**
     * Get balance of address
     */
    getBalanceOfAddress(address) {
        let balance = 0;
        
        for (const block of this.chain) {
            for (const trans of block.transactions) {
                if (trans.from === address) {
                    balance -= trans.amount;
                }
                if (trans.to === address) {
                    balance += trans.amount;
                }
            }
        }
        
        return balance;
    }
    
    /**
     * Display the entire chain
     */
    displayChain() {
        console.log('\n=== BLOCKCHAIN ===\n');
        this.chain.forEach(block => {
            console.log(JSON.stringify(block.display(), null, 2));
            console.log('---');
        });
    }
}
```

### **3.5.4 Adding Proof of Work**

The mining mechanism is already included in the Block class. Let's understand it better:

```
Proof of Work Mechanism:

┌─────────────────────────────────────────────────────────────────┐
│                    MINING PROCESS                               │
│                                                                 │
│  Goal: Find a hash below a certain target value                │
│                                                                 │
│  Target Visualization:                                          │
│  ┌─────────────────────────────────────────────────────────┐   │
│  │ Maximum possible hash:                                   │   │
│  │ FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF │   │
│  │                                                          │   │
│  │ Target (difficulty 4):                                   │   │
│  │ 0000FFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF │   │
│  │                                                          │   │
│  │ Valid hash must be less than target:                     │   │
│  │ 0000A3B2C1D0E9F8... ✓ VALID                              │   │
│  │ 0001A3B2C1D0E9F8... ✗ INVALID (greater than target)      │   │
│  └─────────────────────────────────────────────────────────┘   │
│                                                                 │
│  Process:                                                       │
│  ────────                                                       │
│  1. Start with nonce = 0                                       │
│  2. Calculate block hash                                       │
│  3. Check if hash < target                                     │
│  4. If yes: SUCCESS!                                           │
│  5. If no: increment nonce, repeat                             │
│                                                                 │
│  Probability of success:                                        │
│  • Difficulty 1: 1 in 16 (hex digit)                          │
│  • Difficulty 2: 1 in 256                                      │
│  • Difficulty 3: 1 in 4096                                     │
│  • Difficulty 4: 1 in 65536                                    │
│                                                                 │
│  Adjustable difficulty ensures consistent block time           │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **3.5.5 Validating the Chain**

```javascript
/**
 * Transaction Class for our blockchain
 */
class Transaction {
    constructor(from, to, amount, type = 'regular') {
        this.from = from;        // null for coinbase
        this.to = to;
        this.amount = amount;
        this.type = type;        // 'coinbase' or 'regular'
        this.timestamp = Date.now();
        this.hash = this.calculateHash();
        
        // For UTXO tracking (simplified)
        this.fromTxId = null;
        this.fromOutputIndex = 0;
    }
    
    calculateHash() {
        return sha256(
            this.from + this.to + this.amount + this.timestamp + this.type
        );
    }
    
    isValid(utxoPool) {
        // Coinbase transactions are always valid
        if (this.type === 'coinbase') return true;
        
        // Check if sender has funds (simplified)
        if (!this.from) return false;
        
        // In real implementation, verify signature and UTXO existence
        return true;
    }
}
```

### **3.5.6 Complete Code Implementation**

Here's the complete runnable code:

```javascript
// Complete Simple Blockchain Implementation
// Run with: node simple-blockchain.js

const crypto = require('crypto');

// ==================== UTILITY FUNCTIONS ====================

function sha256(data) {
    return crypto.createHash('sha256').update(data).digest('hex');
}

function createMerkleRoot(transactions) {
    if (transactions.length === 0) return sha256('empty');
    if (transactions.length === 1) return transactions[0].hash;
    
    let hashes = transactions.map(tx => tx.hash);
    while (hashes.length > 1) {
        const nextLevel = [];
        for (let i = 0; i < hashes.length; i += 2) {
            if (i + 1 === hashes.length) {
                nextLevel.push(sha256(hashes[i] + hashes[i]));
            } else {
                nextLevel.push(sha256(hashes[i] + hashes[i + 1]));
            }
        }
        hashes = nextLevel;
    }
    return hashes[0];
}

// ==================== TRANSACTION CLASS ====================

class Transaction {
    constructor(from, to, amount, type = 'regular') {
        this.from = from;
        this.to = to;
        this.amount = amount;
        this.type = type;
        this.timestamp = Date.now();
        this.hash = this.calculateHash();
    }
    
    calculateHash() {
        return sha256(
            (this.from || '') + this.to + this.amount + this.timestamp + this.type
        );
    }
    
    isValid() {
        if (this.type === 'coinbase') return true;
        if (!this.from || !this.to) return false;
        if (this.amount <= 0) return false;
        return true;
    }
}

// ==================== BLOCK CLASS ====================

class Block {
    constructor(index, previousHash, transactions, difficulty = 2) {
        this.index = index;
        this.previousHash = previousHash;
        this.transactions = transactions;
        this.timestamp = Date.now();
        this.merkleRoot = createMerkleRoot(transactions);
        this.difficulty = difficulty;
        this.nonce = 0;
        this.hash = this.calculateHash();
    }
    
    calculateHash() {
        return sha256(
            this.index + 
            this.previousHash + 
            this.merkleRoot + 
            this.timestamp + 
            this.difficulty + 
            this.nonce
        );
    }
    
    mineBlock() {
        const target = '0'.repeat(this.difficulty);
        console.log(`⛏️  Mining block ${this.index}...`);
        const startTime = Date.now();
        
        while (this.hash.substring(0, this.difficulty) !== target) {
            this.nonce++;
            this.hash = this.calculateHash();
        }
        
        const duration = ((Date.now() - startTime) / 1000).toFixed(3);
        console.log(`✅ Block mined!`);
        console.log(`   Hash: ${this.hash}`);
        console.log(`   Nonce: ${this.nonce}`);
        console.log(`   Time: ${duration}s\n`);
    }
    
    display() {
        return {
            index: this.index,
            hash: this.hash.substring(0, 16) + '...',
            previousHash: this.previousHash.substring(0, 16) + '...',
            merkleRoot: this.merkleRoot.substring(0, 16) + '...',
            transactions: this.transactions.length,
            nonce: this.nonce,
            difficulty: this.difficulty,
            timestamp: new Date(this.timestamp).toLocaleTimeString()
        };
    }
}

// ==================== BLOCKCHAIN CLASS ====================

class Blockchain {
    constructor(difficulty = 2) {
        this.chain = [this.createGenesisBlock()];
        this.difficulty = difficulty;
        this.pendingTransactions = [];
        this.miningReward = 100;
    }
    
    createGenesisBlock() {
        const genesisTx = new Transaction(null, 'genesis', 0, 'coinbase');
        return new Block(0, '0'.repeat(64), [genesisTx], this.difficulty);
    }
    
    getLatestBlock() {
        return this.chain[this.chain.length - 1];
    }
    
    addTransaction(transaction) {
        if (!transaction.isValid()) {
            throw new Error('Invalid transaction');
        }
        this.pendingTransactions.push(transaction);
        console.log(`📋 Transaction added: ${transaction.hash.substring(0, 16)}...`);
    }
    
    minePendingTransactions(minerAddress) {
        // Include mining reward
        const rewardTx = new Transaction(null, minerAddress, this.miningReward, 'coinbase');
        const allTransactions = [rewardTx, ...this.pendingTransactions];
        
        const block = new Block(
            this.chain.length,
            this.getLatestBlock().hash,
            allTransactions,
            this.difficulty
        );
        
        block.mineBlock();
        this.chain.push(block);
        
        console.log(`🎉 Block ${block.index} added to chain!\n`);
        this.pendingTransactions = [];
        return block;
    }
    
    getBalance(address) {
        let balance = 0;
        for (const block of this.chain) {
            for (const tx of block.transactions) {
                if (tx.from === address) balance -= tx.amount;
                if (tx.to === address) balance += tx.amount;
            }
        }
        return balance;
    }
    
    isChainValid() {
        for (let i = 1; i < this.chain.length; i++) {
            const current = this.chain[i];
            const previous = this.chain[i - 1];
            
            if (current.hash !== current.calculateHash()) {
                console.error(`❌ Block ${i} hash invalid`);
                return false;
            }
            
            if (current.previousHash !== previous.hash) {
                console.error(`❌ Block ${i} previous hash mismatch`);
                return false;
            }
        }
        return true;
    }
    
    display() {
        console.log('\n' + '='.repeat(50));
        console.log('BLOCKCHAIN VISUALIZATION');
        console.log('='.repeat(50));
        this.chain.forEach((block, i) => {
            console.log(`\n🔗 Block ${block.index}`);
            console.log(`   Hash: ${block.hash}`);
            console.log(`   Prev: ${block.previousHash}`);
            console.log(`   Merkle: ${block.merkleRoot}`);
            console.log(`   Tx Count: ${block.transactions.length}`);
            console.log(`   Nonce: ${block.nonce}`);
        });
        console.log('\n' + '='.repeat(50));
    }
}

// ==================== DEMONSTRATION ====================

console.log('🚀 INITIALIZING BLOCKCHAIN\n');
const myCoin = new Blockchain(3); // Difficulty 3

// Create addresses
const alice = 'alice-address-123';
const bob = 'bob-address-456';
const miner = 'miner-address-789';

console.log(`⛏️  Mining difficulty: ${myCoin.difficulty}`);
console.log(`💰 Mining reward: ${myCoin.miningReward} coins\n`);

// Mine first block (genesis already exists, mine block 1 for rewards)
console.log('--- Block 1: Miner gets reward ---');
myCoin.minePendingTransactions(miner);

console.log('--- Creating Transactions ---');
const tx1 = new Transaction(miner, alice, 50);
myCoin.addTransaction(tx1);

const tx2 = new Transaction(miner, bob, 30);
myCoin.addTransaction(tx2);

const tx3 = new Transaction(alice, bob, 10);
myCoin.addTransaction(tx3);

console.log('\n--- Block 2: Mine pending transactions ---');
myCoin.minePendingTransactions(miner);

console.log('--- Block 3: Mine again for more rewards ---');
myCoin.minePendingTransactions(miner);

// Display final state
myCoin.display();

console.log('\n💰 FINAL BALANCES:');
console.log(`Alice: ${myCoin.getBalance(alice)} coins`);
console.log(`Bob: ${myCoin.getBalance(bob)} coins`);
console.log(`Miner: ${myCoin.getBalance(miner)} coins`);

console.log('\n🔒 Chain Validation:', myCoin.isChainValid() ? '✅ VALID' : '❌ INVALID');

// Demonstrate tampering detection
console.log('\n--- TAMPERING TEST ---');
console.log('Attempting to modify Block 1...');
myCoin.chain[1].transactions[0].amount = 1000; // Try to change reward

console.log('🔒 Chain Validation after tampering:', 
    myCoin.isChainValid() ? '✅ VALID' : '❌ INVALID - Tampering detected!');

/*
EXPECTED OUTPUT:

🚀 INITIALIZING BLOCKCHAIN

⛏️  Mining difficulty: 3
💰 Mining reward: 100 coins

--- Block 1: Miner gets reward ---
⛏️  Mining block 1...
✅ Block mined!
   Hash: 000a3f2b1c8d...
   Nonce: 3421
   Time: 0.145s

🎉 Block 1 added to chain!

--- Creating Transactions ---
📋 Transaction added: 7a8b9c0d...
📋 Transaction added: 3e4f5a6b...
📋 Transaction added: 1c2d3e4f...

--- Block 2: Mine pending transactions ---
⛏️  Mining block 2...
✅ Block mined!
   Hash: 0007e8f9a0b1...
   Nonce: 8923
   Time: 0.312s

🎉 Block 2 added to chain!

... (continues)
*/
```

---

## **Chapter Summary**

### **Key Takeaways**

```
┌─────────────────────────────────────────────────────────────────┐
│                    CHAPTER 3 SUMMARY                            │
│                                                                 │
│  1. BLOCK ANATOMY                                               │
│     • Header: Metadata (80 bytes in Bitcoin)                   │
│       - Previous hash, Merkle root, timestamp, nonce           │
│     • Body: Transaction data                                    │
│     • Genesis block: First block, hardcoded                     │
│                                                                 │
│  2. CHAIN STRUCTURE                                             │
│     • Blocks linked via cryptographic hashes                   │
│     • Changing any block breaks entire chain                   │
│     • Height = position, Depth = blocks after                   │
│                                                                 │
│  3. TRANSACTION MODELS                                          │
│     • UTXO (Bitcoin):                                           │
│       - Outputs become inputs to new transactions              │
│       - Must spend entire output (change comes back)           │
│       - Prevents double-spending via consumption                │
│     • Account (Ethereum):                                       │
│       - Global state of balances                                │
│       - Nonce prevents replays                                  │
│       - More intuitive, supports smart contracts               │
│                                                                 │
│  4. TRANSACTION COMPONENTS                                      │
│     • Inputs: References to previous outputs                   │
│     • Outputs: New UTXOs created                               │
│     • Fees: Incentive for miners/validators                    │
│     • Nonce: Sequential number preventing replays              │
│                                                                 │
│  5. MEMORY POOL                                                 │
│     • Temporary holding area for unconfirmed txs               │
│     • Fee market determines priority                            │
│     • Gossip protocol propagates transactions                   │
│                                                                 │
│  6. IMPLEMENTATION                                              │
│     • Built working blockchain from scratch                    │
│     • Proof of Work mining                                      │
│     • Chain validation                                          │
│     • Transaction processing                                    │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### **Practice Questions**

1. What are the main components of a block header and what does each do?

2. Explain the difference between UTXO and Account-based transaction models.

3. Why is the nonce important in Ethereum transactions?

4. How does Proof of Work secure the blockchain?

5. What happens if you try to modify a transaction in an old block?

### **Code Challenges**

1. **Difficulty Adjustment**: Implement automatic difficulty adjustment based on block time.

2. **Transaction Validation**: Add signature verification to the transaction class.

3. **Persistence**: Add functionality to save/load the blockchain to/from a file.

4. **P2P Network**: Simulate multiple nodes and implement chain synchronization.

---

## **Chapter 4 Preview: Consensus Mechanisms**

In the next chapter, we'll explore how distributed networks agree on the state of the blockchain without a central authority:

- **The Byzantine Generals Problem**: Understanding the fundamental challenge of distributed consensus
- **Proof of Work (PoW)**: Deep dive into mining, difficulty adjustment, and 51% attacks
- **Proof of Stake (PoS)**: How Ethereum and other chains validate without energy consumption
- **Other Consensus Mechanisms**: Delegated PoS, Practical Byzantine Fault Tolerance, and more
- **Implementation**: Code examples of different consensus algorithms

We'll understand why consensus is the "holy grail" of blockchain technology and how different approaches trade off between security, decentralization, and efficiency.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='2. cryptography_fundamentals.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='../2. consensus_mechanisms/4. understanding_consensus.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
